## Using Komo for IK

In [1]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry

**ry-c++-log** /home/jung-su/git/robotics-course/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback



In [2]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
D = C.view()

In [3]:
obj = C.addFrame("object")
obj.setPosition([1., 0., 1.5])
obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])

In [4]:
IK = C.komo_IK(False)
IK.add_qControlObjective([], 1, 1.)
IK.addObjective([1.], 
                ry.FS.positionRel, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                [1e2],
                [0.,0.,-.2]);

In [5]:
# Calling the optimizer (True means random initialization/restart)
IK.optimize()
IK.getReport()

{'F_qItself/1-#28': {'order': 1.0, 'type': 'sos', 'sos': 0.782000659610913},
 'F_qQuaternionNorms/0-#84': {'order': 0.0, 'type': 'eq'},
 'F_PositionRel/0-object-R_gripperCenter': {'order': 0.0,
  'type': 'eq',
  'eq': 1.2962742145482264e-05},
 'sos': 0.7898206662070222,
 'ineq': 0.0,
 'eq': 1.2962742145482264e-05,
 'f': 0.0}

In [7]:
C.setFrameState( IK.getFrameState(0) )

In [8]:
# Move object and reoptimize

# move object
obj.setPosition([.2,.2,1.5])

# # copy C into the IK's internal configuration(s)
# IK.setConfigurations(C)

# redefine the IK problem
IK = C.komo_IK(False)
IK.add_qControlObjective([], 1, 1.)
IK.addObjective([1.], 
                ry.FS.positionRel, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                [1e2],
                [0.,0.,-.2]);
# reoptimize
IK.optimize(0.) # 0 indicates: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getFrameState(0) )

{'F_qItself/1-#28': {'order': 1.0, 'type': 'sos', 'sos': 0.643078099141695}, 'F_qQuaternionNorms/0-#84': {'order': 0.0, 'type': 'eq'}, 'F_PositionRel/0-object-R_gripperCenter': {'order': 0.0, 'type': 'eq', 'eq': 6.120705833001505e-05}, 'sos': 0.6495088801331119, 'ineq': 0.0, 'eq': 6.120705833001505e-05, 'f': 0.0}


## Path Optimization

In [9]:
obj.setPosition([.8,.2,1.5])

# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,40, 5., True)
komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "object"], ry.OT.sos, [1e2])
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)
komo.addObjective([], ry.FS.distance, ["R_gripper", "object"], ry.OT.ineq, [1e2])

komo.optimize()
komo.getReport()

{'F_qItself/2-#28': {'order': 2.0, 'type': 'sos', 'sos': 0.1347086451569076},
 'F_PositionDiff/0-R_gripperCenter-object': {'order': 0.0,
  'type': 'sos',
  'sos': 0.0001082374836834694},
 'F_qItself/1-#28': {'order': 1.0, 'type': 'eq', 'eq': 1.0668376972632065e-05},
 'F_PairCollision/0-R_gripper-object': {'order': 0.0, 'type': 'ineq'},
 'sos': 0.13481688264059108,
 'ineq': 0.0,
 'eq': 1.0668376972632065e-05,
 'f': 0.0}

In [10]:
V = komo.view_play(True, 0.1)

In [11]:
C.setFrameState(komo.getFrameState(19))
C.getJointState()
coll = C.feature(ry.FS.accumulatedCollisions, [])
C.computeCollisions() 
coll.eval(C)

(array([0.]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))